In [1]:
import account as ac
import create_account as ca
import write_audio as wa
import scipy.io.wavfile as wav
import sys
from python_speech_features import mfcc
import numpy as np
import pickle
import os
import ltsd
import vad
import verify as verify

accounts = {}
means = None
invstds = None
def normalize(means, invstds, data):
    return (data - means) * invstds

def filter_sig(fs, signal):
    vad_ob = vad.VAD()
    (fs_noise, signal_noise) = wav.read("noise.wav")
    vad_ob.init_noise(fs_noise, signal_noise)
    ret, intervals = vad_ob.filter(fs, signal)
    orig_len = len(signal)
    
    if len(ret) > orig_len/3:
        wav.write('vaded.wav', fs, ret)
        return ret
    return np.array([0])
    #wav.write('vaded.wav', fs, ret)
    #return ret

def main():
    global accounts
    global means
    global invstds
    if means == None and os.path.isfile("allfeatures.txt"):
        means, invstds = ca.recalculate_norm()
    try:
        accounts = pickle.loads(open("accounts.txt", 'rb+').read(),encoding='latin1')
        #os.remove("accounts.txt")
    except IOError:
        print("IO Error")
        pass
    while True:
        call = input("Do you want to login (l) or create a new account (c)? \n")
        if call == "l":
            """
            User decides to log in
            """
            username = input("username: ")
            wa.record_noise()
            wa.write_temp_audio(username)

            wavpath = "temp.wav"
            (rate,sig) = wav.read(wavpath)
            new_sig = filter_sig(rate, sig)
            mfcc_feat = mfcc(new_sig, rate)
            mfcc_feat = np.array(mfcc_feat)

            best_label = ""
            best_ll = -9e99

            for label, account in accounts.items():
                features = normalize(means, invstds, mfcc_feat)
                ll = account.gmm.score_samples(features)[0]
                ll = np.sum(ll)
                print (label, ll)
                if ll > best_ll:
                    best_ll = ll
                    best_label = label
            print (best_ll)
            print (best_label)
            if username != best_label:
                print ("I do not think that account belongs to you")
            else:
                print ("You are now logged in")

        elif call == "c":
            """
            User decides to create a new account
            """
            username = input("Please select a username \n")
            path = "C:/Users/Aayush/Desktop/Minor_2/voice" + "/accounts/" + username + "/"
            if os.path.exists(path):
                print ("Please select another username")
                continue
            verify.generate_key(username)
            wa.record_noise()
            for x in range(3):
                wa.write_audio(username)
            print ("Done recording! Creating account now")
            accounts[username] = ac.Account(username)
            means, invstds = accounts[username].create_account()
            for account in accounts:
                if account != username:
                    accounts[account].fit_gmm(means, invstds)
        #pickle.dump(accounts, open("accounts.txt", 'wb+'))

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print ('Interrupted')
        pickle.dump(accounts, open("accounts.txt", 'wb+'))
        sys.exit(0)
    #except Exception as e:
    
    #   print (e)
    #    print ('Problem with Speech Recognition -- please run in debug mode')
    #    pickle.dump(accounts, open("accounts.txt", 'wb+'))
    #    sys.exit(0)

C:\Users\Aayush\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator GMM from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Do you want to login (l) or create a new account (c)? 
c
Please select a username 
aygrover
Please stay silent as we record the background noise of your current surroundings.
Recording background noise...
Finished recording background noise.
Please read the following words
brushes consolidation bridge war record typewriter collisions growth
Processing...
brushes consolidation bridge war record typewriter collisions growth precious consideration bridge walk with god typewriter collisions throat
77
 Your words did not match.
Please read the following words
speeders book deserts lee percent typists indexes editor
Processing...
speeders book deserts lee percent typists indexes editor speeders book deserts me percent typist indexes editor
96
Recognized String
C:/Users/Aayush/Desktop/Minor_2/voice/accounts/aygrover/0.wav
Please read the following words
secrets memorandum strobe maintenance calendar transmitters engineering halyard installations trial
Processing...
secrets memorandum strobe m

167.473428216 10.8549146066
167.66627188 9.903132903
168.571764446 9.33301460213
169.379891498 6.5853916692
169.766598161 6.4502725734
168.250765135 6.45422692726
167.051835483 6.52075918065
166.489136654 6.55727827445
166.506825238 6.52021030352
166.588074241 6.50442268713
167.014478868 5.23225033566
165.414968041 5.28323042022
167.596213937 5.27986033766
168.50582135 5.29296689085
167.102680436 9.70408941636
168.577442406 13.2619244533
169.204272173 14.7580930764
166.408396598 15.2024033977
169.03504545 15.3175079222
175.135050061 15.3938501611
178.166693437 15.395593483
176.77367312 15.4778228493
175.350456654 16.0373400374
176.524935817 16.2973620391
175.789628765 16.3015656656
174.332520713 16.2684553526
171.403973294 15.9152987918
170.257867223 15.4716507534
170.903988035 15.5543875839
171.36017537 15.0189646853
174.871920007 14.7087434631
176.726834811 14.4538576203
176.004699693 14.4133439127
176.281176796 13.8682888898
177.006465921 13.7331040054
175.765097855 14.6594173755
17

166.207440947 12.4388030905
167.803541509 12.7615828409
166.498827715 12.1931127893
169.016760644 11.9023985367
175.344396523 11.4957679333
173.988608229 11.5017842078
174.064365924 11.5147245254
172.369601665 11.5210667807
174.18773202 11.5134929748
173.215349358 11.4907457604
169.329053651 10.7888436686
166.365040016 10.1403578545
166.618776532 9.86471998042
165.336255242 9.5192269586
166.393043275 7.92595837536
166.592034676 5.31348771855
165.287673996 5.09633481413
165.670148121 5.05209573551
165.834674919 4.96772712984
165.424100604 5.0299234289
165.505257631 5.08582064727
166.15340962 5.09009499658
166.07851261 5.11142261709
165.84655724 5.12411632131
165.298467688 5.13417517564
166.663022804 5.13616495582
165.257560344 5.12918924848
165.29371481 0
164.745218037 0
164.935112994 0
165.584811345 0
165.458570626 0


170.409224616 0
165.920279231 0
166.939419455 0
166.49926137 0
167.332172178 0
166.659841254 9.66994538563
165.47694435 5.95670408082
169.556819823 5.9817020916
167.390848849 6.00970891998
166.115386832 5.98362329502
166.961266743 6.0416590995
165.323446147 5.99615683939
165.033197356 5.95590786367
166.45954816 5.77251496098
166.687860509 5.7556273952
167.869642408 5.71045542399
166.708468093 5.69338409471
167.008286764 5.68604863166
165.83671129 5.58143326485
164.896487298 5.53974433014
167.217459414 5.58656402194
166.156731064 5.57400778529
164.58950058 5.59500417907
166.590501403 5.61490303404
166.035320682 5.58958883714
166.008980425 5.65074721117
167.074402422 5.72138911053
165.780448299 5.7134775046
167.876913415 5.71723948534
166.987813771 5.75947332512
166.291553626 5.75011182478
167.977565717 5.72740864906
167.060874962 5.71627615195
167.041365575 5.70593220964
167.014279635 5.66981831267
166.053827716 5.61305349749
166.859665335 5.55194729586
167.579294643 5.52994420338
167.3

...

175.11021076 20.5109227863
175.135319524 20.5115037452
174.728732947 19.9537788509
175.003378712 14.3362115994
174.277766993 11.806758095
174.668324853 12.715797377
173.481226097 13.1025317085
173.663974817 12.9645726196
172.355271939 12.8071072036
169.995262936 12.7007655532
170.471482848 12.6299685579
170.256036385 12.5550104148
169.081646985 12.4860881406
167.665068353 12.4166854955
168.23334604 12.3651434388
165.789843323 12.3182790203
167.901976741 11.7747912925
166.351201991 10.1843668549
167.483513337 8.11546745222
168.172953615 8.26737593697
167.743693396 8.19652508787
166.23214015 9.18809629671
167.363072991 9.85528439524
172.118261292 10.5180463977
173.705235172 11.2113574311
172.759773063 11.6740236131
175.741200573 12.2241102865
176.08679456 12.5457790566
174.737362143 13.22299041
175.333020905 13.3919784864
176.555428274 13.6994409445
175.696830787 13.8009329138
174.959119467 13.6881634104
175.481566701 13.6018251028
175.309506324 13.470620194
174.480930917 13.3754824982
1

170.981551121 9.69642620104
171.541771202 10.3536206246
172.289766616 10.4765668891
172.431510268 10.7348715453
175.07760743 10.9246381716
174.526408277 11.3338586502
173.853073721 11.5710393035
174.249619004 11.8130778965
173.958492958 12.0023316905
174.165010758 12.0709470892
174.202063758 11.9367753852
174.567220227 11.6635614577
174.716723386 11.5118303634
174.253214936 11.431826051
173.800305931 11.333794096
172.236362109 11.0231761804
171.126768864 10.7335080691
170.545293778 10.4233431525
170.019758416 9.9945642409
166.266125206 8.78373958093
166.908602735 7.95833052159
167.578789433 7.18684626275
169.22466545 6.07524990988
167.632914651 5.70749228885
167.985217252 5.60725132823
167.662126434 5.81995179717
166.350007469 9.06432320557
168.561113461 10.2616744788
169.378311783 10.7654796405
166.919075626 10.9597912457
170.018928481 11.1491434812
175.526283384 11.3726587725
174.714282427 11.6828266556
174.020205172 11.740688926
174.197804828 11.997299321
174.520000828 12.2143317019

165.084182386 5.51067417064
167.297119772 5.4887665343
166.961507679 5.50406359867
166.634983484 0
167.828996618 0
166.61455593 0
166.690975572 0
167.526419664 0
C:/Users/Aayush/Desktop/Minor_2/voice/accounts/aygrover/aygrover.txt


C:\Users\Aayush\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:57: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)


ValueError: GMM estimation with 10 components, but got only 3 samples